In [1]:
import numpy as np
import pandas as pd

import re
import html
import requests

from datetime import datetime, date
import plotly.express as px
import plotly.graph_objs as go



def returncolname(row, colnames):
    if sum(row[['Q1', 'Q2','Q3','Q4']]) != 1:
        return colnames[np.argmax(row[['Q1_Prob', 'Q2_Prob','Q3_Prob','Q4_Prob']].values)]

def probabilityF(row):
    if row['colmax'] == 'Q1_Prob':
        return row.Q1_Prob
    elif row['colmax'] == 'Q2_Prob':
        return row.Q2_Prob
    elif row['colmax'] == 'Q3_Prob':
        return row.Q3_Prob
    elif row['colmax'] == 'Q4_Prob':
        return row.Q4_Prob

    


def Q1(row):
    if (row['Q1'] == 1.0) or (row['colmax'] == 'Q1_Prob'):
        return 1.0
    else:
        return 0.0
    
def Q2(row):
    if (row['Q2'] == 1.0) or (row['colmax'] == 'Q2_Prob'):
        return 1.0
    else:
        return 0.0

def Q3(row):
    if (row['Q3'] == 1.0) or (row['colmax'] == 'Q3_Prob'):
        return 1.0
    else:
        return 0.0
    
def Q4(row):
    if (row['Q4'] == 1.0) or (row['colmax'] == 'Q4_Prob'):
        return 1.0
    else:
        return 0.0
    
def q(row):
    if row['Q1 Posts'] == 1.0:
        return 'Q1'
    if row['Q2 Posts'] == 1.0:
        return 'Q2'
    if row['Q3 Posts'] == 1.0:
        return 'Q3'
    if row['Q4 Posts'] == 1.0:
        return 'Q4'
    else:
        return 'unk'
    

    
    

def buildout():
    """[Provides a final set of features showing which salary range applies to each data.]

    Returns:
        [csv]: [a table containing our original original, predicted, and bracket columns]
    """
    ### changed from app/
    df = pd.read_csv('../app/data/ml.csv')
    df['DatePosted'] = df['DatePosted'].astype('datetime64')
    df = df.sort_values(by="DatePosted")


    #df.replace(np.nan, 0, inplace = True)
    
    df['colmax'] = df.apply(lambda x: returncolname(x, df[['Q1_Prob', 'Q2_Prob','Q3_Prob','Q4_Prob']].columns), axis=1)
    df['Probability'] = df.apply(lambda x: probabilityF(x), axis=1)
    df['Q1 Posts'] = df.apply( lambda row : Q1(row), axis = 1)
    df['Q2 Posts'] = df.apply( lambda row : Q2(row), axis = 1)
    df['Q3 Posts'] = df.apply( lambda row : Q3(row), axis = 1)
    df['Q4 Posts'] = df.apply( lambda row : Q4(row), axis = 1)
    df['Q'] = df.apply( lambda row : q(row), axis = 1)

    
    df.Probability = df.Probability.fillna(1.0)
    # Saving final results table
    df.to_csv('../app/data/graph_ready.csv', index=False)
    
    # Creating and saving only observed salary data
    dfSal = df[df.Salary.notnull()]
    dfSal = dfSal[['DatePosted', 'Salary', 'Q']]
    dfSal.to_csv('../app/data/dfSal.csv', index=False)
    return df

In [2]:
df = buildout()

In [3]:
df.head(1)

,Company,JobTitle,Location,Requirements,Summary,Description,JobUrl,Salary,State,City,...,Q3_Prob,Q4_Pred,Q4_Prob,colmax,Probability,Q1 Posts,Q2 Posts,Q3 Posts,Q4 Posts,Q
10750,alldus,"data scientist, computational biology","Boston, MA",none,a team of data scientists and data engineers ...,job title data scientist computational biolo...,https://www.indeed.com/rc/clk?jk=6c66cc0931e32...,NaN,MA,boston,...,0.18,0.0,0.0,Q3_Prob,0.18,0.0,0.0,1.0,0.0,Q3


In [4]:
from sklearn import metrics

In [5]:
df.columns

Index(['Company', 'JobTitle', 'Location', 'Requirements', 'Summary',
       'Description', 'JobUrl', 'Salary', 'State', 'City', 'DatePosted',
       'Schedule', 'Role', 'Focus', 'Q1', 'Q2', 'Q3', 'Q4', 'Q1_Pred',
       'Q1_Prob', 'Q2_Pred', 'Q2_Prob', 'Q3_Pred', 'Q3_Prob', 'Q4_Pred',
       'Q4_Prob', 'colmax', 'Probability', 'Q1 Posts', 'Q2 Posts', 'Q3 Posts',
       'Q4 Posts', 'Q'],
      dtype='object')

In [6]:
dfc = df[df.Salary.notnull()]

In [7]:
dfProbs = dfc[['Q1', 'Q2', 'Q3', 'Q4',
       'Q1_Pred', 'Q1_Prob', 'Q2_Pred', 'Q2_Prob', 'Q3_Pred', 'Q3_Prob',
       'Q4_Pred', 'Q4_Prob','Salary']]
dfProbs

,Q1,Q2,Q3,Q4,Q1_Pred,Q1_Prob,Q2_Pred,Q2_Prob,Q3_Pred,Q3_Prob,Q4_Pred,Q4_Prob,Salary
20767,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,177500.000000
20770,0.0,0.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,135571.428571
20776,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,180000.000000
20729,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,157500.000000
20728,0.0,0.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,121325.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21669,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31285.714286
21668,0.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,82500.000000
21667,0.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,74500.000000
21666,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,208571.428571


In [8]:
df2 = df[['Q1_Pred', 'Q2_Pred', 'Q3_Pred', 'Q4_Pred']]

In [9]:
df2

,Q1_Pred,Q2_Pred,Q3_Pred,Q4_Pred
10750,0.0,0.0,0.0,0.0
10649,0.0,0.0,0.0,0.0
10650,0.0,0.0,1.0,0.0
10651,0.0,1.0,0.0,0.0
10652,0.0,0.0,0.0,1.0
...,...,...,...,...
19309,0.0,0.0,0.0,0.0
19310,0.0,0.0,1.0,0.0
19312,0.0,0.0,0.0,0.0
19297,0.0,0.0,1.0,1.0


In [10]:
df.columns

Index(['Company', 'JobTitle', 'Location', 'Requirements', 'Summary',
       'Description', 'JobUrl', 'Salary', 'State', 'City', 'DatePosted',
       'Schedule', 'Role', 'Focus', 'Q1', 'Q2', 'Q3', 'Q4', 'Q1_Pred',
       'Q1_Prob', 'Q2_Pred', 'Q2_Prob', 'Q3_Pred', 'Q3_Prob', 'Q4_Pred',
       'Q4_Prob', 'colmax', 'Probability', 'Q1 Posts', 'Q2 Posts', 'Q3 Posts',
       'Q4 Posts', 'Q'],
      dtype='object')

In [11]:
df[['JobTitle', 'Company', 'Location', 'Summary', 'JobUrl', 'Requirements',
       'Description', 'State', 'DatePosted','Salary','Probability','Q'] ].reset_index()

,index,JobTitle,Company,Location,Summary,JobUrl,Requirements,Description,State,DatePosted,Salary,Probability,Q
0,10750,"data scientist, computational biology",alldus,"Boston, MA",a team of data scientists and data engineers ...,https://www.indeed.com/rc/clk?jk=6c66cc0931e32...,none,job title data scientist computational biolo...,MA,2021-04-14,NaN,0.18,Q3
1,10649,data scientist,paylocity,Remote,enthusiastic about advanced analytics and how ...,https://www.indeed.com/rc/clk?jk=1f42de63029f6...,none,don’t just land a job launch your future our ...,Remote,2021-04-14,NaN,0.49,Q2
2,10650,data scientist - media technologies,apple,"Santa Clara Valley, CA",you will collaborate with a team of engineers ...,https://www.indeed.com/rc/clk?jk=82dc473bdbe44...,none,summaryposted oct 28 2020weekly hours 40rol...,CA,2021-04-14,NaN,1.00,Q3
3,10651,full time opportunities for phd students and r...,microsoft,"Redmond, WA",if you need assistance and or a reasonable acc...,https://www.indeed.com/rc/clk?jk=22a79f927d8b4...,none,artificial intelligence ai is dramatically t...,WA,2021-04-14,NaN,0.54,Q2
4,10652,data scientist,ableto,New York State,under eads there are 3 highly collaborative w...,https://www.indeed.com/rc/clk?jk=bb38870d28991...,none,ableto is the leading provider of high quality...,NY,2021-04-14,NaN,0.85,Q4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21738,19309,data infrastructure product manager- vice pres...,"jpmorgan chase bank, n.a.","Jersey City, NJ",4 years of experience as a product manager or...,https://www.indeed.com/rc/clk?jk=2b82eceb1a2cc...,none,about j p morgan corporate investment bankj...,NJ,2021-07-28,NaN,0.42,Q3
21739,19310,"senior director, computational biology - oncol...",astrazeneca,"Gaithersburg, MD 20878+1 location",familiarity with the latest approaches in ai ...,https://www.indeed.com/rc/clk?jk=6a9e0428efe3c...,none,senior director computational biology within ...,MD,2021-07-28,NaN,0.71,Q3
21740,19312,principal product manager,"amazon dev center u.s., inc.","East Palo Alto, CA",amazon web services aws is looking for a sea...,https://www.indeed.com/rc/clk?jk=bfa39b8295bae...,none,bs or ba degree10 years of experience with a...,CA,2021-07-28,NaN,0.49,Q4
21741,19297,manager/director of product manager (bi/ml),experian,"Scottsdale, AZ",experience in data science mathematical model...,https://www.indeed.com/rc/clk?jk=c277d86bf6733...,none,company descriptionexperian is looking for a l...,AZ,2021-07-28,NaN,0.90,Q3


In [12]:
a = df[df.Salary.isnull()]

In [13]:
b= df[df.Salary.notnull()]

In [14]:
df[df.Probability.isnull()]

,Company,JobTitle,Location,Requirements,Summary,Description,JobUrl,Salary,State,City,...,Q3_Prob,Q4_Pred,Q4_Prob,colmax,Probability,Q1 Posts,Q2 Posts,Q3 Posts,Q4 Posts,Q


In [15]:
z = pd.concat([a,b])

In [16]:
za = z[['Salary', 'Company', 'Location', 'Summary', 'JobUrl', 'Requirements',
       'Description', 'State', 'DatePosted','Probability','Q']]
zb = z[['Salary', 'Q1', 'Q2', 'Q3', 'Q4', 'Q1_Pred',
       'Q1_Prob', 'Q2_Pred', 'Q2_Prob', 'Q3_Pred', 'Q3_Prob', 'Q4_Pred',
       'Q4_Prob', 'colmax', 'Q1 Posts', 'Q2 Posts', 'Q3 Posts',
       'Q4 Posts', 'Probability', 'Q']]

In [17]:
za

,Salary,Company,Location,Summary,JobUrl,Requirements,Description,State,DatePosted,Probability,Q
10750,NaN,alldus,"Boston, MA",a team of data scientists and data engineers ...,https://www.indeed.com/rc/clk?jk=6c66cc0931e32...,none,job title data scientist computational biolo...,MA,2021-04-14,0.18,Q3
10649,NaN,paylocity,Remote,enthusiastic about advanced analytics and how ...,https://www.indeed.com/rc/clk?jk=1f42de63029f6...,none,don’t just land a job launch your future our ...,Remote,2021-04-14,0.49,Q2
10650,NaN,apple,"Santa Clara Valley, CA",you will collaborate with a team of engineers ...,https://www.indeed.com/rc/clk?jk=82dc473bdbe44...,none,summaryposted oct 28 2020weekly hours 40rol...,CA,2021-04-14,1.00,Q3
10651,NaN,microsoft,"Redmond, WA",if you need assistance and or a reasonable acc...,https://www.indeed.com/rc/clk?jk=22a79f927d8b4...,none,artificial intelligence ai is dramatically t...,WA,2021-04-14,0.54,Q2
10652,NaN,ableto,New York State,under eads there are 3 highly collaborative w...,https://www.indeed.com/rc/clk?jk=bb38870d28991...,none,ableto is the leading provider of high quality...,NY,2021-04-14,0.85,Q4
...,...,...,...,...,...,...,...,...,...,...,...
21669,31285.714286,wisecode,Remote•Remote,a bachelor's or advanced degree in nutrition ...,https://www.indeed.com/company/WISEcode/jobs/F...,bachelor's (preferred)data entry: 1 year (pref...,wisecode food and nutrition information analy...,Remote,2021-07-28,1.00,Q1
21668,82500.000000,"outsource consultants, llc","Denver, CO 80204 (Central West Denver area)•Re...",bs or higher in data science computer science...,https://www.indeed.com/company/Outsource-Consu...,bachelor's (preferred),sales operations and reporting analyst mn or ...,Remote,2021-07-28,1.00,Q2
21667,74500.000000,university of colorado boulder,"Boulder, CO 80309 (Colorado University area)",bachelor’s or graduate degree in remote sensin...,https://www.indeed.com/rc/clk?jk=e4a4d5b12811c...,none,job summarythe cooperative institute for resea...,CO,2021-07-28,1.00,Q2
21666,208571.428571,faaz consulting,"Washington, DC•Temporarily Remote",experience with probabilistic modelling clust...,https://www.indeed.com/company/FAAZ-Consulting...,none,experience using pythonexperience with probabi...,Remote,2021-07-28,1.00,Q4


In [18]:
zb

,Salary,Q1,Q2,Q3,Q4,Q1_Pred,Q1_Prob,Q2_Pred,Q2_Prob,Q3_Pred,Q3_Prob,Q4_Pred,Q4_Prob,colmax,Q1 Posts,Q2 Posts,Q3 Posts,Q4 Posts,Probability,Q
10750,NaN,NaN,NaN,NaN,NaN,0.0,0.00,0.0,0.01,0.0,0.18,0.0,0.00,Q3_Prob,0.0,0.0,1.0,0.0,0.18,Q3
10649,NaN,NaN,NaN,NaN,NaN,0.0,0.00,0.0,0.49,0.0,0.01,0.0,0.21,Q2_Prob,0.0,1.0,0.0,0.0,0.49,Q2
10650,NaN,NaN,NaN,NaN,NaN,0.0,0.00,0.0,0.00,1.0,1.00,0.0,0.00,Q3_Prob,0.0,0.0,1.0,0.0,1.00,Q3
10651,NaN,NaN,NaN,NaN,NaN,0.0,0.07,1.0,0.54,0.0,0.02,0.0,0.11,Q2_Prob,0.0,1.0,0.0,0.0,0.54,Q2
10652,NaN,NaN,NaN,NaN,NaN,0.0,0.00,0.0,0.00,0.0,0.01,1.0,0.85,Q4_Prob,0.0,0.0,0.0,1.0,0.85,Q4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21669,31285.714286,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,1.0,0.0,0.0,0.0,1.00,Q1
21668,82500.000000,0.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,0.0,1.0,0.0,0.0,1.00,Q2
21667,74500.000000,0.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,0.0,1.0,0.0,0.0,1.00,Q2
21666,208571.428571,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,0.0,0.0,0.0,1.0,1.00,Q4


In [19]:
c = df2[df2.Salary.isnull()]

AttributeError: 'DataFrame' object has no attribute 'Salary'

In [ ]:
d = df2[df2.Salary.notnull()]

In [ ]:
zz = pd.concat([c,d])
zz

In [ ]:
a = sum(df['Q1 Posts'] ==1)
a

In [ ]:
b = sum(df['Q2 Posts'] ==1)
b

In [ ]:
c = sum(df['Q3 Posts'] ==1)
c

In [ ]:
d = sum(df['Q4 Posts'] ==1)
d

In [ ]:
sum([a,b,c,d])

In [ ]:
dd = pd.read_csv('../app/data/ml.csv')

In [ ]:
dd.columns

In [20]:
dd

NameError: name 'dd' is not defined